In [ ]:
import os
import json
import pandas as pd
import traceback

In [ ]:
from langchain.chat_models import ChatOpenAI

In [ ]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables

In [ ]:
KEY=os.getenv("OPENAI_API_KEY")

In [ ]:
llm=ChatOpenAI(openai_api_key=KEY, model="gpt-3.5-turbo", temperature=0.5)

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [ ]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
     "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },
     "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        "correct": "correct answer",
    },

}

In [ ]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it is your job to 
create a quiz of {number} multiple choice questions for {subject} students in {tone}
tone. Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESONSE_JSON and use it as a guide.
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
"""

In [ ]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [ ]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [ ]:
TEMPLATE2 = """
You are an expert english grammerian and writer. Given a Multiple Choice Quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity,
If the quiz is not at per with the cognitive abilities and analytical abilities of the students.
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities.
Quiz_MCQs:
{quiz}

Check from an expert Egnlish writer of the above quiz:
"""

In [ ]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables=["subject", "quiz"], template=TEMPLATE2
)

In [ ]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [ ]:
generate_evaluation_chain = SequentialChain(
    chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"], verbose=True )  

In [ ]:
file_path=r"C:\Users\QC\mcqgen\data.txt"

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()


In [ ]:
print(TEXT)

In [ ]:
# Serialize the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)


In [ ]:
NUMBER = 5
SUBJECT = "Computer Science"
TONE = "professional"

In [ ]:
#How to setup Token usage Tracking in Langchain

with get_openai_callback() as cb:
    response=generate_evaluation_chain (
        {
            "text": TEXT,
            "number": NUMBER ,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

In [ ]:
print(f"Total Tokens Used: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: ${cb.total_cost}")

In [ ]:
quiz=response.get("quiz")

In [ ]:
quiz=json.loads(quiz)

In [ ]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
        f"{option}: {option_value}"
        for option, option_value in value["options"].items()
        ]
    )
    corect = value["correct"]
    quiz_table_data.append({
        "Question": mcq,
        "Options": options,
        "Correct Answer": corect
    })

In [ ]:
quiz_table_data

In [ ]:
quiz= pd.DataFrame(quiz_table_data)

In [ ]:
quiz.to_csv("machinelearning_quiz.csv", index=False)